In [1]:
import sys
sys.path.append("../")
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from utils.inkml2img import convert_dir
from tqdm.auto import tqdm
from data.dataset import Im2LatexDataset
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from model.vit import ViT
import lightning as L
import wandb
from transformers import PreTrainedTokenizerFast
from tokenizers import Tokenizer, pre_tokenizers
from tokenizers.models import BPE
from model.decoder import Decoder, DecoderAttention, DecoderTransformerBlock
from model.model import Img2MathModel


In [2]:
device = torch.device('mps')
img_dims = [256, 256]
data = Im2LatexDataset(path_to_data="../data/",
                       tokenizer="../data/tokenizer.json", block_size=32, img_dims=img_dims, batch_size=1, device=device)
batch = next(iter(data.train))
img, label = batch
vocab_size = len(data.tokenizer.get_vocab())
#model = Img2MathModel.load_from_checkpoint('../train/img2math/362edsd8/checkpoints/epoch=9-step=65180.ckpt', map_location=device)
model = Img2MathModel(10, 32, 2160, 0, img_dims, 16, num_heads=2).to(device)
model.training_step(batch, 1)
#model.eval()
#logits, _ = model(img)
#torch.argmax(logits)
#val_imgs = img
#model.training_step(batch, 1)

torch.Size([1, 32, 10]) tensor([[[ 1.8602, -1.7757, -0.4856,  0.3035, -0.3928, -0.1712, -1.0683,
           1.2496,  0.0723,  0.4081],
         [ 1.8533, -1.7807, -0.4830,  0.3080, -0.4076, -0.1653, -1.0573,
           1.2619,  0.0747,  0.3960],
         [ 1.8644, -1.7697, -0.4828,  0.3050, -0.3896, -0.1743, -1.0882,
           1.2317,  0.0846,  0.4189],
         [ 1.8424, -1.7945, -0.4692,  0.3055, -0.3958, -0.1585, -1.0812,
           1.2375,  0.0867,  0.4271],
         [ 1.8504, -1.7846, -0.4864,  0.3131, -0.4047, -0.1716, -1.0642,
           1.2451,  0.0875,  0.4154],
         [ 1.8518, -1.7806, -0.4874,  0.3155, -0.4064, -0.1590, -1.0753,
           1.2393,  0.0859,  0.4161],
         [ 1.8586, -1.7793, -0.4924,  0.3128, -0.3979, -0.1742, -1.0630,
           1.2432,  0.0820,  0.4101],
         [ 1.8525, -1.7855, -0.4863,  0.3128, -0.4014, -0.1725, -1.0614,
           1.2468,  0.0875,  0.4076],
         [ 1.8557, -1.7789, -0.4900,  0.3150, -0.4027, -0.1569, -1.0750,
           1.23

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/core/module.py:407: UserWarning: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`
  rank_zero_warn(


tensor(7.6332, device='mps:0', grad_fn=<NllLossBackward0>)

In [3]:
def decode(tokenizer, sequence):
    dec = [tokenizer.decode(tok) for tok in sequence]
    return ''.join([detok.replace('Ġ', ' ') for detok in dec])

#preds = [model.generate(img) for img in val_imgs]
tokenizer = PreTrainedTokenizerFast(tokenizer_file='../data/tokenizer.json', padding_side='left')

seq = model.generate(val_imgs[0])
decode(tokenizer, seq)



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 #preds = [model.generate(img) for img in val_imgs]                                          │
│    6 tokenizer = PreTrainedTokenizerFast(tokenizer_file='../data/tokenizer.json', padding_sid    │
│    7                                                                                             │
│ ❱  8 seq = model.generate(val_imgs[0])                                                           │
│    9 decode(tokenizer, seq)                                                                      │
│   10                                                                                             │
│   11                                                                                             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'val_imgs' is not defined

In [3]:
TOKENIZER_FILE = '../data/tokenizer.json'

def decode(tokenizer, sequence):
    dec = [tokenizer.decode(tok) for tok in sequence]
    return ''.join([detok.replace('Ġ', ' ') for detok in dec])

class ImagePredictionLogger(L.Callback):
    
    def __init__(self, val_samples, num_samples=10):
        super().__init__()
        self.tokenizer = PreTrainedTokenizerFast(tokenizer_file=TOKENIZER_FILE, padding_side='left')
        self.val_imgs, self.val_labels = val_samples
        self.val_imgs = self.val_imgs[:num_samples]
        self.val_labels = [decode(self.tokenizer, label) for label in self.val_labels['input_ids'][:num_samples]]


    def on_validation_epoch_end(self, trainer, pl_module):
        val_imgs = self.val_imgs.to(device=pl_module.device)

        pred_tokens = [pl_module.generate(x) for x in val_imgs]
        preds = [decode(self.tokenizer, pred) for pred in pred_tokens]

        trainer.logger.experiment.log({
            "examples": [wandb.Image(x, caption=f"Pred:{pred}, Label:{y}")
                         for x, pred, y in zip(val_imgs, preds, self.val_labels)],
            "global_step": trainer.global_step
        })

logger = WandbLogger(project='img2math')

trainer = L.Trainer(limit_train_batches=10, max_epochs=1, log_every_n_steps=10, deterministic=True,
                    logger=logger, accelerator='mps', callbacks=[ImagePredictionLogger(batch)])

trainer.fit(model, data.train, data.test)
wandb.finish()


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: baolong. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name    | Type    | Params
------------------------------------
0 | encoder | ViT     | 8.9 M 
1 | decoder | Decoder | 14.9 M
------------------------------------
23.9 M    Trainable params
0         Non-trainable params
23.9 M    Total params
95.443    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/dylanh/miniconda3/envs/pytorch/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


epoch,▁▁
global_step,▁█
train/loss,▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/loss_epoch,▁
val/loss_step,▂▄▃▃▃▂▃▃▆▂▄▂▂▂▃▃█▁▂▂▁▃▄▃▂▆▂▂▁▃▃▂▄▃▃▂▇▃▃▁
epoch,0
global_step,10
train/loss,0.00864
trainer/global_step,9
val/loss_epoch,0.44301


In [17]:
device = torch.device('mps')
a = torch.randn(256, 512).to(device)

for i in range(100):
    b = torch.randn(256, 512).to(device)
    c = a @ b.T
    print(f"mem usage: {torch._C._mps_currentAllocatedMemory()}")


mem usage: 1310720
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1835008
mem usage: 1572864
mem usage: 1